# Project Open Data BCN: Cycling

https://opendata-ajuntament.barcelona.cat/es/

##1. Instalamos Pymongo

In [ ]:
!pip install pymongo[srv]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.1/677.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.4/305.4 kB 30.7 MB/s eta 0:00:00


In [ ]:
# Import Libraries

import pymongo                            # Library to access MongoDB
from pymongo import MongoClient           # Imports MongoClient
import pandas as pd                       # Library to work with dataframes
import folium                             # Library to visualize a map

En MongoDB Atlas, desde la pestaña de Overview, nos aparecerá nuestros Database Deployments. En el que queramos interactuar, pulsamos en Connect y pulsando en Drivers (y seleccionando Python) nos aparece lo siguiente:


```
Connecting with MongoDB Driver
1. Select your driver and version

We recommend installing and using the latest driver version.
Driver
Version
2. Install your driver
Run the following on the command line
Note: Use appropriate Python 3 executable

python -m pip install "pymongo[srv]"==3.11

View MongoDB Python Driver installation instructions.
3. Add your connection string into your application code

View full code sample

mongodb+srv://big_data_technology_icai:<password>@bigdatatechnologyicai.et6ildu.mongodb.net/?retryWrites=true&w=majority

Replace <password> with the password for the big_data_technology_icai user. Ensure any option params are URL encoded

```


.

In [ ]:
# Parámetro de conexión proporcionados en la celda anterior:

uri = 'mongodb+srv://somethingtoconnecttothecluster'

# Start client to connect to MongoDB server
client = MongoClient( uri )

In [ ]:
db = client.big_data_tech_icai         # Set the database to work on
db.list_collection_names()             # List the collections available

['cycling']

In [ ]:
collection = db.cycling                # Collection alias

In [ ]:
# Counts the documents in database

num_documents = collection.count_documents({'_id' : {'$exists' : 1}})
print ( 'Number of documents in database = ' + str(num_documents) )
list ( collection.find().limit(1) )

Number of documents in database = 926


[{'_id': ObjectId('65c161322168b07b594b702e'),
  'id': 1,
  'type': 'BIKE',
  'latitude': 41.397952,
  'longitude': 2.180042,
  'streetName': 'Gran Via Corts Catalanes',
  'streetNumber': 760,
  'altitude': 21,
  'slots': 2,
  'bikes': 25,
  'nearbyStations': '24, 369, 387, 426',
  'status': 'OPN',
  'updateTime': '01/08/18 17:43:08'}]

##2. Visualizamos los datos

In [ ]:
# Recuperar un documento de la colección y mostrar su estructura
documento = collection.find_one()
list ( documento )

['_id',
 'id',
 'type',
 'latitude',
 'longitude',
 'streetName',
 'streetNumber',
 'altitude',
 'slots',
 'bikes',
 'nearbyStations',
 'status',
 'updateTime']

In [ ]:
# Mostrar los tipos de datos de cada variable
# De cara a los cálculos de agregación o queries, corroboramos que cada variable tiene el tipo deseado

print("\nTipos de datos de cada variable:")
print("")
for key, value in documento.items():
    print(f"{key}: {type(value)}")


Tipos de datos de cada variable:

_id: <class 'bson.objectid.ObjectId'>
id: <class 'int'>
type: <class 'str'>
latitude: <class 'float'>
longitude: <class 'float'>
streetName: <class 'str'>
streetNumber: <class 'int'>
altitude: <class 'int'>
slots: <class 'int'>
bikes: <class 'int'>
nearbyStations: <class 'str'>
status: <class 'str'>
updateTime: <class 'str'>


## Ejemplos

**Contar el número total de estaciones de bicicletas:**

In [ ]:
# Contar el número total de estaciones de bicicletas
result_total_estaciones = collection.aggregate([
    {"$group": {"_id": None, "total": {"$sum": 1}}}
])

# Obtener el número total de estaciones de bicicletas
total_estaciones = result_total_estaciones.next()["total"]

# Imprimir el resultado
print("Número total de estaciones de bicicletas:", total_estaciones)

Número total de estaciones de bicicletas: 926


Esta consulta agrupa todos los documentos en la colección y cuenta el número total de documentos.

**Contar el número total de estaciones de bicicletas activas:**

In [ ]:
import pandas as pd

# Filtrar estaciones con status "OPN"
filters = {'status': 'OPN'}
fields = {'_id': 1, 'latitude': 1, 'longitude': 1, 'bikes': 1, 'slots': 1}

query = list(collection.find(filters, fields))
df_estaciones_opn = pd.DataFrame(query)

# Imprimir el número de estaciones con status "OPN"
print('Número de estaciones con status "OPN":', len(query))
print("")

# Mostrar el DataFrame con las estaciones con status "OPN"
print(df_estaciones_opn)

Número de estaciones con status "OPN": 920

                          _id   latitude  longitude  slots  bikes
0    65c161322168b07b594b702e  41.397952   2.180042      2     25
1    65c161322168b07b594b702f  41.395530   2.177060      0     27
2    65c161322168b07b594b7030  41.394072   2.183441     19      7
3    65c161322168b07b594b7031  41.393470   2.181490     11      7
4    65c161322168b07b594b7032  41.391075   2.180223     34      2
..                        ...        ...        ...    ...    ...
915  65c161322168b07b594b73c7  41.394232   2.175278     10     14
916  65c161322168b07b594b73c8  41.381860   2.177086      9     14
917  65c161322168b07b594b73c9  41.389481   2.165357     16      8
918  65c161322168b07b594b73ca  41.377191   2.149283     13     11
919  65c161322168b07b594b73cb  41.404871   2.175141     13     11

[920 rows x 5 columns]


**Cantidad total de bicicletas disponibles en todas las estaciones**

In [ ]:
# Obtener la cantidad total de bicicletas disponibles en todas las estaciones
result_total_bicicletas = collection.aggregate([
    {"$group": {"_id": None, "total_bikes": {"$sum": "$bikes"}}}
])

# Convertir el resultado en un DataFrame de pandas
df_total_bicicletas = pd.DataFrame(result_total_bicicletas)

# Imprimir el resultado
print("Cantidad total de bicicletas disponibles en todas las estaciones:")
print(df_total_bicicletas)

Cantidad total de bicicletas disponibles en todas las estaciones:
    _id  total_bikes
0  None         8034


**Encontrar la estación con la mayor cantidad de bicicletas disponibles**

In [ ]:
# Encontrar la estación con la mayor cantidad de bicicletas disponibles
result_max_bicicletas = collection.aggregate([
    {"$sort": {"bikes": -1}},
    {"$limit": 1}
])

# Convertir el resultado en un DataFrame de pandas
df_max_bicicletas = pd.DataFrame(result_max_bicicletas)

# Imprimir el resultado
print("Estación con la mayor cantidad de bicicletas disponibles:")
print(df_max_bicicletas)

Estación con la mayor cantidad de bicicletas disponibles:
                        _id   id  type   latitude  longitude  \
0  65c161322168b07b594b7183  346  BIKE  41.360654   2.139132   

             streetName  streetNumber  altitude  slots  bikes  \
0  Carrer de la Foneria            33        48      0     30   

       nearbyStations status         updateTime  
0  345, 347, 348, 349    OPN  01/08/18 17:43:08  


**Calcular el promedio de bicicletas disponibles por estación**

In [ ]:
# Calcular el promedio de bicicletas disponibles por estación
result_promedio_bicicletas = collection.aggregate([
    {"$group": {"_id": None, "avg_bikes": {"$avg": "$bikes"}}}
])

# Convertir el resultado en un DataFrame de pandas
df_promedio_bicicletas = pd.DataFrame(result_promedio_bicicletas)

# Imprimir el resultado
print("Promedio de bicicletas disponibles por estación:")
print(df_promedio_bicicletas['avg_bikes'])

Promedio de bicicletas disponibles por estación:
0    8.676026
Name: avg_bikes, dtype: float64


**Número de estaciones de bicicletas activas con al menos 10 bicicletas**

In [ ]:
# Loading database query in pandas Dataframe
filters = {'status':'OPN', 'bikes' : {'$gte' : 10 }}   # Usage of gte Query Operator  $gte = "greater than or equal"
fields = { '_id', 'latitude' , 'longitude', 'bikes', 'slots'}

query = list( collection.find( filters , fields ) )
df = pd.DataFrame ( query )                             # Load the database reply in a Pandas DataFrame

In [ ]:
print ( 'Numer of active stations with at least 10 bicycles: ' + str(len (query)) )

Numer of active stations with at least 10 bicycles: 369


In [ ]:
df.iloc[0] # prints the first DataFrame row

_id          65c161322168b07b594b702e
latitude                    41.397952
longitude                    2.180042
slots                               2
bikes                              25
Name: 0, dtype: object

**Encontrar las estaciones ubicadas en una cierta latitud y longitud**

In [ ]:
# Encontrar las estaciones ubicadas en una cierta latitud y longitud
result_estaciones_lat_long = collection.aggregate([
    {"$match": {"latitude": {"$gte": 41.0, "$lte": 42.0}, "longitude": {"$gte": 2.17, "$lte": 2.19}}}
])

# Convertir el resultado en un DataFrame de pandas
df_estaciones_lat_long = pd.DataFrame(result_estaciones_lat_long)

# Imprimir el resultado
print("Estaciones ubicadas en una cierta latitud y longitud:")
print("")
print(df_estaciones_lat_long)

Estaciones ubicadas en una cierta latitud y longitud:

                          _id   id           type   latitude  longitude  \
0    65c161322168b07b594b702e    1           BIKE  41.397952   2.180042   
1    65c161322168b07b594b702f    2           BIKE  41.395530   2.177060   
2    65c161322168b07b594b7030    3           BIKE  41.394072   2.183441   
3    65c161322168b07b594b7031    4           BIKE  41.393470   2.181490   
4    65c161322168b07b594b7032    5           BIKE  41.391075   2.180223   
..                        ...  ...            ...        ...        ...   
283  65c161322168b07b594b73c4  488  BIKE-ELECTRIC  41.423976   2.184391   
284  65c161322168b07b594b73c5  489  BIKE-ELECTRIC  41.409532   2.188282   
285  65c161322168b07b594b73c7  492  BIKE-ELECTRIC  41.394232   2.175278   
286  65c161322168b07b594b73c8  493  BIKE-ELECTRIC  41.381860   2.177086   
287  65c161322168b07b594b73cb  496  BIKE-ELECTRIC  41.404871   2.175141   

                    streetName streetNumber 

## Visualización de estaciones

Se emplea la librería Folium para crear un mapa interactivo:

* Crear el mapa: Se crea un objeto locationmap que representa el mapa. El parámetro location especifica las coordenadas del centro del mapa, y zoom_start establece el nivel de zoom inicial. width y height establecen el tamaño del mapa en píxeles.

* Recorrer el DataFrame: El código recorre el DataFrame df con un bucle for. La variable longitud contiene la longitud del DataFrame, es decir, el número de filas.

* Obtener coordenadas y descripción: Dentro del bucle, se extraen las coordenadas de longitud y latitud de cada fila del DataFrame. Además, se crea una cadena de descripción que contiene el número de bicicletas y los espacios vacíos en la estación.

* Crear marcador: Para cada fila del DataFrame, se crea un marcador en el mapa utilizando las coordenadas de longitud y latitud. El marcador contiene una ventana emergente con la descripción creada anteriormente. El icono del marcador se establece en rojo.

* Agregar marcador al mapa: Cada marcador se agrega al mapa utilizando el método add_to(locationmap).

* Mostrar el mapa: Finalmente, el mapa se muestra en la celda del notebook.

In [ ]:
center_lat = 41.378
center_lon = 2.139

locationmap = folium.Map(location=[ center_lat , center_lon ], zoom_start=16, width=800, height=600 )
longitud  = len( df )

for i in range ( longitud ):
    lng = float(df.iloc[i]['longitude'])
    lat = float(df.iloc[i]['latitude'])
    description = 'Bikes: ' + str(df.iloc[i]['bikes']) + '<br> Empty slots: ' + str(df.iloc[i]['slots'])
    folium.Marker( [ lat , lng ],
                 popup= description,
                 icon=folium.Icon(color='red')).add_to(locationmap)

locationmap

## Ejercicios

**Contar el número total de estaciones únicas por tipo (type):**

In [ ]:
# Your code here

result_estaciones_tipo = collection.aggregate([
    {
        "$group" : {"_id": "$type", "total_stations": {"$sum": 1}}
    }
])

df_estaciones_tipo = pd.DataFrame(result_estaciones_tipo)

print("Total de estaciones por tipo:")
print(df_estaciones_tipo)

Total de estaciones por tipo:
             _id  total_stations
0  BIKE-ELECTRIC              90
1           BIKE             836


**Encontrar la estación con la menor cantidad de bicicletas disponibles:**

In [ ]:
# Your code here

result_menos_bicis = collection.aggregate([
        {"$match": {"bikes": {"$gte": 1}}},
        {"$sort": {"bikes": 1}  },
        {"$limit": 1}
])

df_menos_bicis = pd.DataFrame(result_menos_bicis)

print("Estacion con menos bicis:")
print(df_menos_bicis)

Estacion con menos bicis:
                        _id  id  type   latitude  longitude streetName  \
0  65c161322168b07b594b703e  18  BIKE  41.406086   2.174167  RossellĂ³   

   streetNumber  altitude  slots  bikes    nearbyStations status  \
0           453        49     24      1  19, 28, 120, 370    OPN   

          updateTime  
0  01/08/18 17:43:08  


**Calcular el promedio de bicicletas disponibles por cada tipo de estación (type):**

In [ ]:
# Your code here

result_avg_bicis = collection.aggregate([
    {"$group": {"_id":"$type", "avg_available_bikes": {"$avg": "$bikes"}} }
])


df_avg_bicis = pd.DataFrame(result_avg_bicis)

print("Media de tipos de bicis por estación")
print(df_avg_bicis)

Media de tipos de bicis por estación
             _id  avg_available_bikes
0  BIKE-ELECTRIC             5.188889
1           BIKE             9.051435


**Encontrar las estaciones que están a menos de 1 km de una estación dada (utilizando la fórmula de Haversine para calcular distancias):**

La función geodesic de la librería geopy se utiliza para calcular la distancia entre dos puntos en la superficie de la Tierra utilizando la fórmula del haversine. Esta fórmula tiene en cuenta la curvatura de la Tierra y proporciona una mayor precisión en distancias largas en comparación con la distancia euclidiana.

La fórmula del haversine es una ecuación trigonométrica que calcula la distancia entre dos puntos en una esfera (en este caso, la Tierra) dados sus valores de latitud y longitud. La fórmula utiliza las funciones trigonométricas seno y coseno para calcular la distancia arcoss la superficie de la Tierra.

Un ejemplo de cómo se utiliza la función geodesic con geopy:

In [ ]:
from geopy.distance import geodesic

# Coordenadas de dos puntos (latitud, longitud)
coordenada_1 = (41.397952, 2.180042)  # Barcelona, España
coordenada_2 = (40.712776, -74.005974)  # Nueva York, EE. UU.

# Calcular la distancia entre los dos puntos
distancia = geodesic(coordenada_1, coordenada_2).kilometers

print("La distancia entre Barcelona y Nueva York es:", round(distancia,2), "kilómetros")

La distancia entre Barcelona y Nueva York es: 6181.72 kilómetros


**Encontrar las estaciones que están a menos de un 1km de una estación**

In [ ]:
result_station_location = collection.aggregate([
      {"$match": {"id": 1}}
  ])

df = pd.DataFrame(result_station_location)
print(df["latitude"][0])
print(df["longitude"][0])

41.397952
2.180042
